In [1]:
import sys
import pickle

from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

HOME = Path.cwd().parent
sys.path.append(str(HOME))

from src.data import BatteryData
from src.label import RULLabelAnnotator
from src.feature import BatLiNetFeatureExtractor as FeatureExtractor


In [2]:
data_path = HOME / 'data/processed'
datasources = ['MATR', 'HUST', 'CALCE', 'HNEI', 'RWTH', 'SNL', 'UL_PUR']
batteries = [
    BatteryData.load(x) for ds in datasources
    for x in tqdm(
        list((data_path / ds).glob('*.pkl')),
        desc=f'Loading from data source {ds}')
]

Loading from data source UL_PUR: 100%|██████████| 10/10 [00:00<00:00, 57.25it/s]


In [3]:
mat2bat = defaultdict(list)
labeler = RULLabelAnnotator()
feat_extractor = FeatureExtractor(smooth_features=False)
for bat in tqdm(batteries, desc='Processing features and labels'):
    label = labeler.process_cell(bat)
    if label.isnan():
        continue
    bat.label = label
    bat.feature = feat_extractor.process_cell(bat)
    mat2bat[bat.cathode_material].append(bat)

for mat in mat2bat:
    print(mat, len(mat2bat[mat]), 'cells')

Processing features and labels: 100%|██████████| 403/403 [00:27<00:00, 14.80it/s]

LFP 275 cells
LCO 37 cells
NMC 69 cells
NCA 22 cells


In [5]:
for bat in mat2bat.values():
    for battery in bat:
        battery.cycle_data = None

In [6]:
target = HOME / 'cache/transfer.pkl'
target.parent.mkdir(exist_ok=True)
with open(target, 'wb') as f:
    pickle.dump(mat2bat, f)

In [4]:
# Try to load back
with open(HOME / 'cache/transfer.pkl', 'rb') as f:
    mat2bat = pickle.load(f)